In [1]:
import os
from Bio import SeqIO
import subprocess

## Download rice (Oryza sativa) PPI from STRING

We download physical interaction pairs for *Oryza sativa* from the STRING database version 12, along with their full protein sequences.
Following the same strategy as PIPR (Cheng et al. 2019), we use CD-HIT (Fu et al., 2012) to decrease sequence redundancy of the datasets, in which two PPIs are considered similar if they share a sequence identity greater than 40%.

References:
Chen,M. et al. (2019) Multifaceted protein–protein interaction prediction based on Siamese residual RCNN. Bioinformatics, 35, i305–i314.
Fu,L. et al. (2012) CD-HIT: accelerated for clustering the next-generation sequencing data. Bioinformatics, 28, 3150–3152

In [2]:
# https://cn.string-db.org/
# High confidence (0.7)
# Include AB pairs only
spe = "rice"
link_file = "39947.protein.physical.links.v12.0.min700.onlyAB.tsv"
seq_file = "39947.protein.sequences.v12.0.fa"

ppi_file = os.path.join(spe, "STRING", link_file)
fasta_file = os.path.join(spe, "STRING", seq_file)

filtered_ppi = os.path.join(spe, "action.tsv")
filtered_fasta = os.path.join(spe, "seq.tsv")

## Step 1: Exclude those with < 50 amino acids

In [3]:
protein_sequences = {
    record.id: record
    for record in SeqIO.parse(fasta_file, "fasta")
    if len(record.seq) >= 50
}

print(f"Proteins after length filtering (>= 50 amino acids): {len(protein_sequences)}")

positive_ppis = []
with open(ppi_file, "r") as fin:
    for i, line in enumerate(fin):
        if i == 0:
            continue
        parts = line.strip().split("\t")
        if len(parts) < 2:
            continue
        protein1, protein2 = parts[0], parts[1]
        positive_ppis.append((protein1, protein2))

temp_fasta = os.path.join(spe, "temp_proteins.fasta")
with open(temp_fasta, "w") as fout:
    for pid, record in protein_sequences.items():
        fout.write(f">{pid}\n{record.seq}\n")



Proteins after length filtering (>= 50 amino acids): 43240


In [ ]:
# Step 2: Run CD-HIT clustering to identify proteins with >= 40% sequence identity
output_cluster = os.path.join(spe, "clustered_proteins")

cdhit_cmd = [
    "cd-hit",
    "-i", temp_fasta,
    "-o", output_cluster,
    "-c", "0.4",  # Sequence identity threshold (40%)
    "-n", "2",  # Word length
    "-M", "32000",  # Memory limit (MB)
    "-T", "0"      # Use all CPU cores
]

try:
    subprocess.run(cdhit_cmd, check=True)
    print("CD-HIT clustering completed successfully")
except Exception as e:
    print(f"CD-HIT: {e}")

In [4]:
# Step 3: Process CD-HIT results to exclude proteins with >= 40% sequence identity
# CD-HIT creates clusters where only the representative (first protein in each cluster) is kept
# We need to identify which proteins are representatives (kept) vs clustered (excluded)

representative_proteins = set()
cluster_file = os.path.join(spe, "clustered_proteins.clstr")

with open(cluster_file, "r") as fin:
    current_cluster = []
    for line in fin:
        line = line.strip()
        if line.startswith(">Cluster"):
            # Process previous cluster
            if current_cluster:
                # First protein in cluster is representative, others are excluded
                representative_proteins.add(current_cluster[0])
            current_cluster = []
        elif line:
            # Extract protein ID from line like "0	1234aa, >protein_id... *"
            if "*" in line:
                # This is the representative protein
                protein_id = line.split(">")[1].split("...")[0]
                current_cluster.insert(0, protein_id)
            else:
                # This is a clustered protein
                protein_id = line.split(">")[1].split("...")[0]
                current_cluster.append(protein_id)
    
    # Process last cluster
    if current_cluster:
        representative_proteins.add(current_cluster[0])

print(f"Representative proteins (kept): {len(representative_proteins)}")

# Filter protein_sequences to keep only representatives
filtered_protein_sequences = {
    pid: record for pid, record in protein_sequences.items()
    if pid in representative_proteins
}

print(f"Proteins after sequence identity filtering (>= 40% excluded): {len(filtered_protein_sequences)}")
print(f"Total proteins excluded: {len(protein_sequences) - len(filtered_protein_sequences)}")


Representative proteins (kept): 26408
Proteins after sequence identity filtering (>= 40% excluded): 26408
Total proteins excluded: 16832


In [5]:
# Step 4: Filter PPIs to include only interactions between filtered proteins
filtered_positive_ppis = []
for protein1, protein2 in positive_ppis:
    if protein1 in filtered_protein_sequences and protein2 in filtered_protein_sequences:
        filtered_positive_ppis.append((protein1, protein2))

print(f"Original PPIs: {len(positive_ppis)}")
print(f"Filtered PPIs (both proteins meet criteria): {len(filtered_positive_ppis)}")
print(f"PPIs excluded: {len(positive_ppis) - len(filtered_positive_ppis)}")

# Summary of filtering results
print("\n=== FILTERING SUMMARY ===")
print(f"Proteins with < 50 amino acids: EXCLUDED")
print(f"Proteins with >= 40% sequence identity: EXCLUDED")
print(f"Final protein count: {len(filtered_protein_sequences)}")
print(f"Final PPI count: {len(filtered_positive_ppis)}")


Original PPIs: 27720
Filtered PPIs (both proteins meet criteria): 13178
PPIs excluded: 14542

=== FILTERING SUMMARY ===
Proteins with < 50 amino acids: EXCLUDED
Proteins with >= 40% sequence identity: EXCLUDED
Final protein count: 26408
Final PPI count: 13178


In [6]:
# Step 5: Save filtered data to files
# Save filtered PPIs to action.tsv
with open(filtered_ppi, "w") as fout:
    # Write filtered PPIs
    for protein1, protein2 in filtered_positive_ppis:
        fout.write(f"{protein1}\t{protein2}\t1\n")

print(f"Filtered PPIs saved to: {filtered_ppi}")

# Extract unique protein IDs from filtered PPIs
ppi_protein_ids = set()
for protein1, protein2 in filtered_positive_ppis:
    ppi_protein_ids.add(protein1)
    ppi_protein_ids.add(protein2)

print(f"Unique proteins in filtered PPIs: {len(ppi_protein_ids)}")

# Save only sequences of proteins that appear in filtered PPIs to seq.tsv (TSV format)
with open(filtered_fasta, "w") as fout:
    # Write sequences only for proteins that appear in PPIs
    for protein_id in ppi_protein_ids:
        if protein_id in filtered_protein_sequences:
            record = filtered_protein_sequences[protein_id]
            fout.write(f"{protein_id}\t{str(record.seq)}\n")

print(f"Sequences of PPI proteins saved to: {filtered_fasta}")
print(f"Files saved successfully!")


Filtered PPIs saved to: rice\action.tsv
Unique proteins in filtered PPIs: 2816
Sequences of PPI proteins saved to: rice\seq.tsv
Files saved successfully!
